In [3]:
import inflect

inflector = inflect.engine()

def is_plural(word):
    return inflector.singular_noun(word) is not False

def singular(word):
    if is_plural(word):
        return inflector.singular_noun(word)
    return word

def plural(word):
    if is_plural(word):
        return word
    return inflector.plural(word)

def table_name_to_class_name(table_name: str) -> str:
    """
    Convert a snake_case table name (typically plural) to a PascalCase class name (singular).
    Example: 'user_profiles' -> 'UserProfile'
    """
    parts: list[str] = table_name.split('_')

    if parts:
        last_word = singular(parts[-1])
        parts[-1] = last_word # type: ignore

    return ''.join(word.capitalize() for word in parts)


In [5]:
print(table_name_to_class_name("cats_and_dogs"))
print(table_name_to_class_name("cat"))

CatsAndDog
Cat


In [31]:
print(singular("cats"))
print(singular("cat"))


cat
cat


In [23]:
inflector.singular_noun("attendees") #=> False


'attendee'

In [13]:
print(inflector.plural(inflector.singular_noun("cat") , 2))    # "cats"
print(inflector.plural(inflector.singular_noun("cats"), 2))    # "cat"
print(inflector.plural(inflector.singular_noun("cat" ), 1))    # "cat"

TypeCheckError: argument "text" (bool) is not an instance of inflect.Word

In [6]:
import sys
sys.path.append('/Users/dan/Code/Python/pg_scaffold')
from app.core.db import SessionLocal
from app.crud import conferences, attendees, posters, judges, judging_assignments, criteria, scores, score_criteria
from app.schemas.conferences import ConferencesCreate
from app.schemas.attendees import AttendeesCreate
from app.schemas.posters import PostersCreate
from app.schemas.judges import JudgesCreate
from app.schemas.criteria import CriteriaCreate
from app.schemas.scores import ScoresCreate
import datetime

In [7]:
db = SessionLocal()

## 1. Create Conference

In [10]:
conference = ConferencesCreate(
    name="Comprehensive Test Conference", 
    location="Virtual", 
    start_date=datetime.date(2025, 9, 1), 
    end_date=datetime.date(2025, 9, 5)
)

db_conference = conferences.conferences.create(db, obj_in=conference)
print(f"Created conference: {db_conference.name}")

Created conference: Comprehensive Test Conference


## 2. Create Attendees

In [ ]:
attendee1 = AttendeesCreate(
    first_name="Alice", 
    last_name="Smith", 
    email="alice.smith@example.com", 
    conference_id=db_conference.id
)
db_attendee1 = attendees.create(db, obj_in=attendee1)
print(f"Created attendee: {db_attendee1.first_name} {db_attendee1.last_name}")

attendee2 = AttendeesCreate(
    first_name="Bob", 
    last_name="Johnson", 
    email="bob.johnson@example.com", 
    conference_id=db_conference.id
)
db_attendee2 = attendees.create(db, obj_in=attendee2)
print(f"Created attendee: {db_attendee2.first_name} {db_attendee2.last_name}")

## 3. Create Poster

In [ ]:
poster = PostersCreate(
    board_location="B2", 
    poster_title="A Study of Test-Driven Development", 
    creator_id=db_attendee1.id
)
db_poster = posters.create(db, obj_in=poster)
print(f"Created poster: {db_poster.poster_title}")

## 4. Create Judge

In [ ]:
judge = JudgesCreate(person_id=db_attendee2.id, accepted=True)
db_judge = judges.create(db, obj_in=judge)
print(f"Created judge from attendee: {db_attendee2.first_name} {db_attendee2.last_name}")

## 5. Create Judging Assignment

In [ ]:
assignment = JudgingAssignmentsCreate(judge_id=db_judge.id, poster_id=db_poster.id)
db_assignment = judging_assignments.create(db, obj_in=assignment)
print(f"Created judging assignment for judge {db_judge.id} and poster {db_poster.id}")

## 6. Create Criteria

In [ ]:
criteria1 = CriteriaCreate(name="Clarity", max_score=10)
db_criteria1 = criteria.create(db, obj_in=criteria1)
print(f"Created criteria: {db_criteria1.name}")

criteria2 = CriteriaCreate(name="Methodology", max_score=15)
db_criteria2 = criteria.create(db, obj_in=criteria2)
print(f"Created criteria: {db_criteria2.name}")

## 7. Create Score

In [ ]:
score = ScoresCreate(assignment_id=db_assignment.id, comment="Good work!")
db_score = scores.create(db, obj_in=score)
print(f"Created score for assignment {db_assignment.id}")

## 8. Create Score Criteria

In [ ]:
score_crit1 = ScoreCriteriaCreate(score_id=db_score.id, criterion_id=db_criteria1.id, score=8)
db_score_crit1 = score_criteria.create(db, obj_in=score_crit1)
print(f"Created score criteria for score {db_score.id} and criteria {db_criteria1.name}")

score_crit2 = ScoreCriteriaCreate(score_id=db_score.id, criterion_id=db_criteria2.id, score=12)
db_score_crit2 = score_criteria.create(db, obj_in=score_crit2)
print(f"Created score criteria for score {db_score.id} and criteria {db_criteria2.name}")

## 9. Read, Update, and Delete

In [ ]:
# Read
print("--- Reading Data ---")
print(conferences.get(db, id=db_conference.id))
print(attendees.get(db, id=db_attendee1.id))
print(posters.get(db, id=db_poster.id))
print(judges.get(db, id=db_judge.id))
print(judging_assignments.get(db, id=db_assignment.id))
print(criteria.get(db, id=db_criteria1.id))
print(scores.get(db, id=db_score.id))
print(score_criteria.get(db, id=db_score_crit1.id))

# Update
print("--- Updating Data ---")
updated_poster = posters.update(db, db_obj=db_poster, obj_in={"poster_title": "An Updated Study of TDD"})
print(f"Updated poster title: {updated_poster.poster_title}")

# Delete
print("--- Deleting Data ---")
score_criteria.remove(db, id=db_score_crit1.id)
score_criteria.remove(db, id=db_score_crit2.id)
scores.remove(db, id=db_score.id)
criteria.remove(db, id=db_criteria1.id)
criteria.remove(db, id=db_criteria2.id)
judging_assignments.remove(db, id=db_assignment.id)
judges.remove(db, id=db_judge.id)
posters.remove(db, id=db_poster.id)
attendees.remove(db, id=db_attendee1.id)
attendees.remove(db, id=db_attendee2.id)
conferences.remove(db, id=db_conference.id)
print("Data deleted.")

# Verify Deletion
print("--- Verifying Deletion ---")
print(conferences.get(db, id=db_conference.id))
print(attendees.get(db, id=db_attendee1.id))
print(posters.get(db, id=db_poster.id))